In [2]:
import os
import json

import cv2
import numpy as np
import pandas as pd 
from glob import glob
from tqdm import tqdm
from PIL import Image
import albumentations as A
import matplotlib.pyplot as plt
import matplotlib.patches as patches
plt.rc('font', family='NanumGothicCoding')

In [3]:
class Info:
    def __init__(self):
        self.train = self.get_data('train.json')
        self.val = self.get_data('val.json')
    
    def get_data(self, f_name):
        with open(f_name, mode='r') as f:
            _json = json.load(f)
        ann = pd.DataFrame(_json['annotations'])
        img = pd.DataFrame(_json['images'])
        df = pd.merge(ann, img, left_on='image_id', right_on='id').drop(['id_y'], axis=1).rename(columns={'id_x':'id'})
        return df
    
    def plot(self, img_id):
        fig, ax = plt.subplots(1, 1, figsize=(8, 8))
        ax.axis('off')

        try:
            self._plot(self.train, img_id, ax)
            fig.suptitle(f'"{img_id}" is in train', size=20)
        except:
            self._plot(self.val, img_id, ax)
            fig.suptitle(f'"{img_id}" is in val', size=20)

    def _plot(self, _df, img_id, ax):
        df = _df.query(f'image_id == "{img_id}"')
        assert len(df), 'this "image id" is not in dataframe'
        path = df['file_name'].values[0]
        img = np.array(Image.open(path))
        ax.imshow(img)
        for bbox in df['bbox']:
            ax.add_patch(patches.Rectangle(
                (bbox[0], bbox[1]), bbox[2], bbox[3], edgecolor='seagreen', 
                fill=False, lw=3
            ))
        ax.set_title(df['file_name'].values[0].split('/')[-2], size=20)
info = Info()


In [32]:
info.train[info.train['file_name'].str.contains('진라면')].drop_duplicates('category_id')

,id,image_id,bbox,area,category_id,iscrowd,file_name,height,width
3466,3467,15829_00_m_10,"[570.0, 998.0, 1371.0, 1060.0]",1453260.0,20,0,Training/image/15829_오뚜기진라면매운맛컵110G/15829_00_m...,2988,2988
3792,3793,15831_00_m_10,"[503.0, 987.0, 1370.0, 1085.0]",1486450.0,22,0,Training/image/15831_오뚜기진라면순한맛컵110G/15831_00_m...,2988,2988
10372,10373,30103_00_m_11,"[236.0, 1056.0, 1246.0, 930.0]",1158780.0,60,0,Training/image/30103_오뚜기진라면순한맛120G/30103_00_m_...,2988,2988
24240,24241,60115_00_m_10,"[303.0, 771.0, 995.0, 1181.0]",1175095.0,136,0,Training/image/60115_오뚜기진라면컵순65G/60115_00_m_10...,2988,2988
24870,24871,60118_00_m_10,"[182.0, 908.0, 1078.0, 1269.0]",1367982.0,139,0,Training/image/60118_오뚜기진라면매운맛65G(작은용기)/60118_...,2988,2988
35160,35161,90142_00_m_10,"[696.0, 987.0, 1181.0, 1291.0]",1524671.0,193,0,Training/image/90142_오뚜기진라면매운맛(봉지)120G/90142_0...,2988,2988


In [21]:
print(img_path)
print(bboxes)

Training/image/60111_농심김치사발면86G/60111_60_m_12.jpg
[[1912.0, 775.0, 855.0, 676.0], [1060.0, 1034.0, 850.0, 622.0], [206.0, 1240.0, 894.0, 620.0], [2334.0, 1544.0, 2.0, 0.0]]


In [19]:
transform = A.Compose([
    A.Resize(height=1024, width=1024)],
    bbox_params=A.BboxParams(
        format='coco', 
        min_visibility=0.1, 
        label_fields=[]
        )
    )
    
img2boxes = info.train.groupby('file_name')['bbox'].apply(list)[15239:]


for img_path, bboxes in tqdm(img2boxes.iteritems(), total=len(img2boxes)):
    img = cv2.imread(img_path)

    # transform
    transformed = transform(image=img, bboxes=bboxes)

  0%|          | 1/8128 [00:00<33:51,  4.00it/s]


ValueError: y_max is less than or equal to y_min for bbox (0.7811244979919679, 0.5167336010709505, 0.7817938420348058, 0.5167336010709505).

In [13]:
info.train.sort_values('height')[0::10000]

,id,image_id,bbox,area,category_id,iscrowd,file_name,height,width
13331,13332,35076_30_m_18,"[372.0, 197.0, 444.0, 780.0]",346320.0,74,0,Training/image/35076_피피이씨음성생면)홍면435.2G/35076_3...,1080,1080
33839,33840,90135_60_m_19,"[1740.0, 1211.0, 457.0, 1561.0]",713377.0,186,0,Training/image/90135_오뚜기미역국라면(봉지)115G/90135_60...,2988,2988
5067,5068,15840_60_m_22,"[1274.0, 518.0, 1181.0, 1185.0]",1399485.0,29,0,Training/image/15840_농심사천짜파게티컵115G/15840_60_m_...,2988,2988
15149,15150,35226_30_m_2,"[1508.0, 1068.0, 1337.0, 1065.0]",1423905.0,85,0,Training/image/35226_하늘처럼)오라이시70G/35226_30_m_2...,2988,2988


In [17]:
def resize(df, re_width, re_height):

    img2boxes = df.groupby('file_name')['bbox'].apply(list)
    
    re_df = df.copy().drop(['bbox', 'width', 'height', 'area'], axis=1)
    
    re_df['width'] = re_width
    re_df['height'] = re_height

    transform = A.Compose([
        A.Resize(height=re_height, width=re_width)],
        bbox_params=A.BboxParams(
            format='coco', 
            min_visibility=0.1, 
            label_fields=[]
            )
        )
    for img_path, bboxes in tqdm(img2boxes.iteritems(), total=len(img2boxes)):
        img = cv2.imread(img_path)

        # transform
        transformed = transform(image=img, bboxes=bboxes)
        re_img, re_bboxes = transformed['image'], transformed['bboxes']
        re_df.loc[re_df['file_name']  == img_path, 'bbox'] = pd.Series(re_bboxes).values
        
        os.makedirs('re'+'/'.join(img_path.split('/')[:-1]), exist_ok=True)
        cv2.imwrite('re'+img_path, re_img)

    re_df['area'] = re_df['bbox'].apply(lambda x: round(x[2]*x[3], 2))

    return re_df

def make_json(json_path, df):
    with open(json_path, mode='r') as f:
        _json = json.load(f)

    _json['images'] = []
    _json['annotations'] = []

    for _, row in df.iterrows():
        _json['images'].append({
            'file_name': 're'+row['file_name'],
            'height': row['height'],
            'width': row['width'],
            'id': row['image_id']
        })
        _json['annotations'].append({
            'id' : row['id'],
            'image_id' : row['image_id'],
            'bbox':row['bbox'],
            'area' : row['area'],
            'category_id':row['category_id'],
            'iscrowd' : 0,
        })
    with open('re'+json_path, mode='w') as f:
        json.dump(_json, f)

In [16]:
info = Info()
# re_train = resize(info.train)
# make_json('train.json', re_train)

re_val = resize(info.val)
make_json('val.json', re_val)


100%|██████████| 4938/4938 [00:00<00:00, 14494.49it/s]


In [19]:
with open('reval.json', mode='r') as f:
    temp = json.load(f)

# ------------------------------------------------------------------------------------

In [139]:
for img_path, bboxes in tqdm(img2boxes_train.iteritems(), total=len(img2boxes_train)):
    img = cv2.imread(img_path)
    # img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # # test visualization
    # fig, ax = plt.subplots(1, 1, figsize=(8, 8))
    # ax.axis('off')
    # ax.imshow(img)
    # for bbox in bboxes:
    #     ax.add_patch(patches.Rectangle(
    #         (bbox[0], bbox[1]), bbox[2], bbox[3], edgecolor='seagreen', 
    #         fill=False, lw=3
    #     ))
    
    # transform
    transformed = transform(image=img, bboxes=bboxes)
    re_img, re_bboxes = transformed['image'], transformed['bboxes']
    re_train.loc[re_train['file_name']  == img_path, 'bbox'] = pd.Series(re_bboxes).values
    
    os.makedirs('re'+'/'.join(img_path.split('/')[:-1]), exist_ok=True)
    cv2.imwrite('re'+img_path, re_img)
    
    # # test visualization
    # fig, ax = plt.subplots(1, 1, figsize=(8, 8))
    # ax.axis('off')
    
    # ax.imshow(trans_img)
    # for bbox in trans_bboxes:
    #     ax.add_patch(patches.Rectangle(
    #         (bbox[0], bbox[1]), bbox[2], bbox[3], edgecolor='seagreen', 
    #         fill=False, lw=3
    #     ))


re_train['area'] = re_train['bbox'].apply(lambda x: round(x[2]*x[3], 2))

  0%|          | 79/23367 [00:11<54:39,  7.10it/s]


KeyboardInterrupt: 

In [101]:
import json
from glob import glob
from tqdm import tqdm
import xml.etree.ElementTree as elemTree

In [37]:
data_type='Training'

img_folders = sorted(glob(f'{data_type}/image/*'))
label_folders = sorted(glob(f'{data_type}/label/*'))

In [102]:
temp_json = {
    'images':[],
    'categories':[],
    'annotations':[]
}

#categories
for _id, cat in enumerate(label_folders):
    cat = cat.split('/')[-1]
    temp_json['categories'].append(
        {
            'supercategory':'Defect',
            'id': _id,
            'name':cat
        }
    )

# images, annotations
ann_id = 0
for label_folder in tqdm(label_folders):
    label_files = sorted(glob(label_folder + '/*'))
    for label_file in label_files:
        if 'meta' in label_file:
            f_name = label_file\
                .replace('label', 'image')\
                .replace('_meta.xml', '.jpg')
            xml = open(label_file, 'rt', encoding='UTF8')
            tree = elemTree.parse(xml)
            root = tree.getroot()
            item_no = root.find('div_cd').find('item_no').text
            annotation = root.find('annotation')
            size = annotation.find('size')
            w = size.find('width').text
            h = size.find('height').text
            img_id = f_name.split('/')[-1].split('.')[0]
            
            temp_json['images'].append({
                'file_name':f_name,
                'height': int(h),
                'width': int(w),
                'id': img_id
            })
            for _object in annotation.findall('object'):
                ann_id +=1
                bbox = _object.find('bndbox')
                xmin = float(bbox.find('xmin').text)
                ymin = float(bbox.find('ymin').text)
                xmax = float(bbox.find('xmax').text)
                ymax = float(bbox.find('ymax').text)
                ann_w = xmax-xmin
                ann_h = ymax-ymin
                temp_json['annotations'].append({
                    'id' : ann_id,
                    'image_id' : img_id,
                    'bbox':[xmin, ymin, ann_w, ann_h],
                    'area' : ann_w * ann_h,
                    'category_id':item_no,
                    'is_crowd' : 0,
                    'segmentation' : 0
                })
            # break
    
    # break

100%|██████████| 205/205 [00:03<00:00, 64.52it/s]


In [ ]:
from PIL import Image
# for folder_img in folder_imgs:
img = sorted(glob(folder_labels[0]+'/*'))[0]\
.replace('label','image').replace('.xml', '.jpg')
print(img)
Image.open(img)